In [15]:
import pandas as pd
import boto3
import json
import sys 
import configparser

In [16]:
config = configparser.ConfigParser()
config.read_file(open('/Users/RahulReddy/dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

## Creating a dictionary with columns Param and Value 

df = pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

In [17]:
import boto3

ec2 = boto3.resource('ec2',region_name='us-west-2',aws_access_key_id=KEY,aws_secret_access_key=SECRET)

s3 = boto3.resource('s3',region_name='us-west-2',aws_access_key_id=KEY,aws_secret_access_key=SECRET)

iam = boto3.client('iam',region_name='us-west-2',aws_access_key_id=KEY,aws_secret_access_key=SECRET)

redshift = boto3.client('redshift',region_name='us-west-2',aws_access_key_id=KEY,aws_secret_access_key=SECRET)

In [4]:
samplebucket = s3.Bucket("awssampledbuswest2")
for i in samplebucket.objects.filter(Prefix='ssbgz'):
    print(i)

s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/customer0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/dwdate.tbl.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0000_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0001_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0002_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0003_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0004_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0005_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0006_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='ssbgz/lineorder0007_part_00.gz')
s3.ObjectSummary(bucket_name='awssampledbuswest2', key='s

In [18]:
dwhRole = iam.create_role(
          Path='/',
          RoleName=DWH_IAM_ROLE_NAME,
          AssumeRolePolicyDocument=json.dumps(
          {'Statement': [{'Action': 'sts:AssumeRole',
                       'Effect': 'Allow',
                       'Principal': {'Service': 'redshift.amazonaws.com'}}],
                        'Version': '2012-10-17'
          }
          ),
          Description='Redshift role to run the select on S3 bucket',
          Tags=[
              {
                  'Key': 'role of the Red',
                  'Value': 'redshiftcluser'
              },
             ]
) 

In [7]:
delete_role = iam.delete_role(RoleName='DWH_IAM_ROLE_NAME')

In [21]:
iam.get_role_policy??

Signature: iam.get_role_policy(*args, **kwargs)
Docstring:
Retrieves the specified inline policy document that is embedded with the specified IAM role.

 

.. note::

   

  Policies returned by this API are URL-encoded compliant with `RFC 3986 <https://tools.ietf.org/html/rfc3986>`__ . You can use a URL decoding method to convert the policy back to plain JSON text. For example, if you use Java, you can use the ``decode`` method of the ``java.net.URLDecoder`` utility class in the Java SDK. Other languages and SDKs provide similar functionality.

   

 

An IAM role can also have managed policies attached to it. To retrieve a managed policy document that is attached to a role, use  GetPolicy to determine the policy's default version, then use  GetPolicyVersion to retrieve the policy document.

 

For more information about policies, see `Managed Policies and Inline Policies <https://docs.aws.amazon.com/IAM/latest/UserGuide/policies-managed-vs-inline.html>`__ in the *IAM User Guide* .

 

In [8]:
#iam.list_policies(OnlyAttached=True)['Policies']
response = iam.get_policy(
    PolicyArn='arn:aws:iam::aws:policy/AWSLambdaExecute'
)
print(response['Policy'])

{'PolicyName': 'AWSLambdaExecute', 'PolicyId': 'ANPAJE5FX7FQZSU5XAKGO', 'Arn': 'arn:aws:iam::aws:policy/AWSLambdaExecute', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'Description': 'Provides Put, Get access to S3 and full access to CloudWatch Logs.', 'CreateDate': datetime.datetime(2015, 2, 6, 18, 40, 46, tzinfo=tzutc()), 'UpdateDate': datetime.datetime(2015, 2, 6, 18, 40, 46, tzinfo=tzutc())}


In [19]:

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']



200

In [21]:
print("1.3 Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)


1.3 Get the IAM role ARN
arn:aws:iam::846457387363:role/dwhRole


In [23]:
import random

In [24]:
x = [random.randint(0,9) for i in range(10)]

 index at 1 data at:5
 index at 2 data at:0
 index at 3 data at:5
 index at 4 data at:5
 index at 5 data at:8
 index at 6 data at:7
 index at 7 data at:8
 index at 8 data at:9
 index at 9 data at:7
 index at 10 data at:0
